In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import jax
import jax.numpy as jnp

from dosipy.bhte import BHTE
from dosipy.constants import eps_0, mu_0
from dosipy.field import poynting
from dosipy.utils.integrate import elementwise_dblquad
from dosipy.utils.derive import holoborodko
from dosipy.utils.dataloader import (load_tissue_diel_properties,
                                     load_antenna_el_properties)
from dosipy.utils.viz import fig_config, set_colorblind

In [ ]:
print(f'platform: {jax.lib.xla_bridge.get_backend().platform}')

In [ ]:
# jax.config.update("jax_enable_x64", True)

In [ ]:
set_colorblind()
%config InlineBackend.figure_format = 'retina'

# 3-D bioheat transfer equation in time

The model for the PSTD method is a 3-D block of skin.
Realistic antenna is considered -- half-wave dipole centrally powered by a voltage source set to 1V.

### Setup

In [ ]:
# frequency
f = 60e9

# conductivity, relative permitivity, tangent loss and penetration depth
sigma, eps_r, tan_loss, pen_depth = load_tissue_diel_properties('skin_dry', f)

# `pen_depth` is the energy penetration depth into tissue, which is defined as
# the distance beneath the surface at which the SAR has fallen to a factor of
# 1/e below that at the surface; one-half of the more commonly reported wave
# penetration depth
pen_depth = pen_depth / 2

# air (vacuum) resistance 
Z_air = np.sqrt(mu_0 / eps_0)

# dry skin resistance
Z_skin_dry = np.sqrt(mu_0 / (eps_r * eps_0))

# energy (Fresnel) transmission coefficient into the tissue
T_tr = 2 * Z_skin_dry / (Z_air + Z_skin_dry)

In [ ]:
# antenna electric properties (Poljak 2005)
antenna_data = load_antenna_el_properties(f)

# antenna position -- coordinates
xs = antenna_data.x.to_numpy()
xs = jnp.asarray(xs)
ys = jnp.zeros_like(xs)
zs = jnp.zeros_like(xs)

# current through the antenna
Is = antenna_data.ireal.to_numpy() + antenna_data.iimag.to_numpy() * 1j
Is_x = holoborodko(Is, xs[1]-xs[0])

# solution domain setup
s_res = 9
target_area = (0.02, 0.02)  # meters x meters
A = target_area[0] * target_area[1]  # meters^2
h = 10 / 1000  # distance from the antenna
xt = jnp.linspace(-target_area[0]/2, target_area[0]/2, s_res) + xs[-1] / 2
yt = jnp.linspace(-target_area[1]/2, target_area[1]/2, s_res)
zt = jnp.linspace(h, h + pen_depth, s_res)

### EM-field

Free space electromagnetic field distribution across the surface of the tissue and approximation of SAR.

In [ ]:
# compute time-averaged complex Poynting vector
Sx = np.empty((xt.size, yt.size), dtype=np.complex128)
Sy = np.empty((xt.size, yt.size), dtype=np.complex128)
Sz = np.empty((xt.size, yt.size), dtype=np.complex128)
for xi, _xt in enumerate(xt):
    for yi, _yt in enumerate(yt):
        _Sx, _Sy, _Sz = poynting(_xt, _yt, h, xs, ys, zs, f, Is)
        Sx[xi, yi] = _Sx
        Sy[xi, yi] = _Sy
        Sz[xi, yi] = _Sz
        
# compute incident power density distribution across the surface
Sinc = 1 / 2 * np.abs(np.sqrt(Sx ** 2 + Sy ** 2 + Sz ** 2))

# compute average incident power density on the surface
Xt, Yt = np.meshgrid(xt, yt)
Sav = 1 / (2 * A) * elementwise_dblquad(points=np.column_stack((Xt.ravel(), Yt.ravel())),
                                        values=Sz.real.ravel(),
                                        degree=9)

# approximate SAR into the skin
rho = 1109.  # skin density in kg/m^3
amplitude = Sinc * T_tr / (rho * pen_depth)
exp_decay = np.exp(-zt / (h + pen_depth))
SAR = np.outer(amplitude, exp_decay).reshape(*amplitude.shape, *exp_decay.shape)

### Simulation

Solution of the 3-D bio-heat transfer equation in time.

In [ ]:
# initialization of the solver
sim_time = 360  # seconds
t_res = 101
domain = (*target_area, pen_depth)
bhte = BHTE(sim_time, t_res, domain, s_res,
            Qm=0, SAR=SAR)

# initial condition setup and solution
T0 = 37.
T = bhte.solve(T0)

# temperature rise on the surface
deltaTsurf = T[-1, :, :, 0] - T[0, :, :, 0]

# heating factor
hf = deltaTsurf.max() / Sav

### Visualization

Temperature change on the surface.

In [ ]:
extent=(xt.min(), xt.max(), yt.min(), yt.max())
interp = 'none'

fig_config(scaler=2)
fig = plt.figure()
ax = fig.add_subplot()
cs = ax.imshow(deltaTsurf, extent=extent, interpolation=interp, cmap='viridis')
cbar = fig.colorbar(cs, ax=ax)
cbar.ax.set_ylabel('$\\Delta T_{sur}$ [°C]')
ax.plot(xs, ys, 'r-', linewidth=4,
        label=(f'$\\lambda/2$ dipole antenna\ndistance = {h * 1000} mm'))
ax.set(xlabel='$x$ [m]',
       ylabel='$y$ [m]',
       xticks=[xt.min(), (xt.min() + xt.max()) / 2, xt.max()],
       yticks=[-0.01, -0.005, 0.0, 0.005, 0.01])
ax.legend()
fig.tight_layout();